In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
# nota las librerias se instalaron en python3 no conda

In [ ]:
raw_datasets = load_dataset("glue", "sst2")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)



In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

raw_datasets['train'].features

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
# enfoquémonos en el modelo. Lo vamos a crear exactamente como lo hicimos en la sección anterior.
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
# Para asegurarnos de que todo va a salir sin problems durante el entrenamiento, 
# vamos a pasar un lote a este modelo:

outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

# Todos los modelos Transformers 🤗 van a retornar la pérdida cuando se pasan los labels,
# y también obtenemos los logits (dos por cada entrada en nuestro lote, 
# asi que es un tensor de tamaño 8 x 2).

Estamos casi listos para escribir nuestro bucle de entrenamiento! Nos están faltando dos cosas: un optimizador y un programador de la rata de aprendizaje. Ya que estamos tratando de replicar a mano lo que el Trainer estaba haciendo, usaremos los mismos valores por defecto. El optimizador usado por el Trainer es AdamW, que es el mismo que Adam, pero con un cambio para la regularización de decremento de los pesos (ver “Decoupled Weight Decay Regularization” por Ilya Loshchilov y Frank Hutter):

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

Finalmente, el programador por defecto de la rata de aprendizaje es un decremento lineal desde al valor máximo (5e-5) hasta 0. Para definirlo apropiadamente, necesitamos saber el número de pasos de entrenamiento que vamos a tener, el cual viene dado por el número de épocas que deseamos correr multiplicado por el número de lotes de entrenamiento (que es el largo de nuestro dataloader de entrenamiento). El Trainer usa tres épocas por defecto, asi que usaremos eso:

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Como lo hicimos anteriormente, usaremos una métrica ofrecida por la libreria 🤗 Evaluate. Ya hemos visto el método metric.compute(), pero de hecho las métricas se pueden acumular sobre los lotes a medida que avanzamos en el bucle de predicción con el método add_batch(). Una vez que hemos acumulado todos los lotes, podemos obtener el resultado final con metric.compute(). Aquí se muestra como se puede implementar en un bucle de evaluación:

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()